In [1]:
# Install required libraries
!pip install groq requests -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 3.3 MB/s eta 0:00:00


In [2]:
import os
import requests
import json
from groq import Groq
from google.colab import userdata

# Configure the Groq client with your API key from Colab secrets
try:
    GROQ_API_KEY = userdata.get('GROQ_API_KEY')
    client = Groq(api_key=GROQ_API_KEY)
    print("Groq client configured successfully.")
except userdata.SecretNotFoundError:
    print("ERROR: 'GROQ_API_KEY' not found. Please add it to Colab secrets.")
except Exception as e:
    print(f"An error occurred: {e}")


# Define the LLM model we'll use
LLM_MODEL = 'llama3-8b-8192'

# ConceptNet API endpoint
CONCEPTNET_API_URL = "http://api.conceptnet.io"

Groq client configured successfully.


In [3]:
# --- Block 1: LLM Answer Generation ---
def get_llm_answer(question):
    """Generates a direct, fluent answer to the user's question."""
    print(f"-> Generating LLM answer for: '{question}'")
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "You are a helpful tutor. Answer the user's question clearly and concisely.",
            },
            {
                "role": "user",
                "content": question,
            }
        ],
        model=LLM_MODEL,
        temperature=0.7,
    )
    return chat_completion.choices[0].message.content

In [4]:
# --- Block 2: Concept Extraction Engine ---
def extract_concepts_with_llm(question, answer):
    """Uses an LLM to extract key concepts from the question and answer."""
    print("-> Extracting key concepts...")
    prompt = f"""
From the following user question and AI answer, extract the most important concepts.
A concept should be one or two words, like "sunscreen" or "ultraviolet rays".
Return the concepts as a JSON object with two keys: "question_concepts" and "answer_concepts".

Question: "{question}"
Answer: "{answer}"

Example:
Question: "Why do we use sunscreen?"
Answer: "We use sunscreen to protect our skin from harmful ultraviolet (UV) rays from the sun."
Output:
{{
  "question_concepts": ["sunscreen", "sun"],
  "answer_concepts": ["protecting skin", "ultraviolet rays", "skin damage"]
}}

Now, provide the output for the given question and answer.
"""
    chat_completion = client.chat.completions.create(
        messages=[
            {"role": "user", "content": prompt}
        ],
        model=LLM_MODEL,
        response_format={"type": "json_object"},
    )
    try:
        concepts = json.loads(chat_completion.choices[0].message.content)
        # Simple normalization: lowercase and replace spaces with underscores for ConceptNet
        concepts['question_concepts'] = [c.lower().replace(' ', '_') for c in concepts.get('question_concepts', [])]
        concepts['answer_concepts'] = [c.lower().replace(' ', '_') for c in concepts.get('answer_concepts', [])]
        return concepts
    except json.JSONDecodeError:
        print("Error: LLM did not return valid JSON for concepts.")
        return {"question_concepts": [], "answer_concepts": []}


In [5]:
# --- Block 3: Relational Path Finding in ConceptNet ---
def find_conceptnet_path(start_concept, end_concept):
    """Finds a relationship path between two concepts in ConceptNet."""
    print(f"-> Finding ConceptNet path between '{start_concept}' and '{end_concept}'...")
    start_uri = f"/c/en/{start_concept}"
    end_uri = f"/c/en/{end_concept}"

    try:
        # We look for relationships where the start concept is the start
        response = requests.get(f"{CONCEPTNET_API_URL}/query?start={start_uri}&end={end_uri}&limit=5")
        response.raise_for_status()
        data = response.json()
        if data['edges']:
            edge = data['edges'][0] # Take the first, most relevant edge
            rel_label = edge['rel']['label']
            return f'"{start_concept}" {rel_label} "{end_concept}"'

        # If no direct path, try reversing the concepts
        response = requests.get(f"{CONCEPTNET_API_URL}/query?start={end_uri}&end={start_uri}&limit=5")
        response.raise_for_status()
        data = response.json()
        if data['edges']:
            edge = data['edges'][0]
            rel_label = edge['rel']['label']
            return f'"{end_uri}" {rel_label} "{start_uri}"' # Note the order is swapped back

    except requests.exceptions.RequestException as e:
        print(f"  - ConceptNet API error: {e}")
    return None

In [6]:
# --- Block 4: Explanation Synthesis and Refinement ---
def build_and_refine_explanation(question, relations):
    """Takes structured relations and uses an LLM to build a natural language explanation."""
    if not relations:
        return "Could not find any commonsense relationships to build an explanation."

    print("-> Refining explanation with LLM...")
    structured_explanation = "\n- ".join(relations)
    prompt = f"""
You are an explanation engine.
Given a user's question and a set of structured commonsense relationships from ConceptNet, synthesize them into a single, easy-to-understand paragraph that explains the reasoning behind the answer.
Do not just list the relationships. Weave them into a coherent narrative.

User Question: "{question}"

Commonsense Relationships:
- {structured_explanation}

Generated Explanation:
"""
    chat_completion = client.chat.completions.create(
        messages=[
            {"role": "user", "content": prompt}
        ],
        model=LLM_MODEL,
        temperature=0.5,
    )
    return chat_completion.choices[0].message.content

In [13]:
def explainable_ai_tutor(user_question):
    """The main pipeline for the explainable AI tutor."""
    print("="*50)
    print(f"USER QUESTION: {user_question}")
    print("="*50)

    # 1. Get LLM Answer
    llm_answer = get_llm_answer(user_question)
    print(f"\n🤖 LLM Answer:\n{llm_answer}\n")

    # 2. Extract Concepts
    concepts = extract_concepts_with_llm(user_question, llm_answer)
    print(f"  - Extracted Question Concepts: {concepts['question_concepts']}")
    print(f"  - Extracted Answer Concepts: {concepts['answer_concepts']}")

    if not concepts['question_concepts'] or not concepts['answer_concepts']:
        print("\nCould not extract sufficient concepts to build an explanation.")
        return

    # 3. Find Relational Paths in ConceptNet
    found_relations = []
    # This is a simple but effective strategy: find links between question & answer concepts
    for q_concept in concepts['question_concepts']:
        for a_concept in concepts['answer_concepts']:
            path = find_conceptnet_path(q_concept, a_concept)
            if path and path not in found_relations:
                found_relations.append(path)

    # Also find paths between answer concepts themselves for more richness
    if len(concepts['answer_concepts']) > 1:
        for i in range(len(concepts['answer_concepts'])):
            for j in range(i + 1, len(concepts['answer_concepts'])):
                path = find_conceptnet_path(concepts['answer_concepts'][i], concepts['answer_concepts'][j])
                if path and path not in found_relations:
                    found_relations.append(path)

    if not found_relations:
      print("\n🧠 Explanation:\nCould not find commonsense relationships in ConceptNet for the extracted concepts.")
      return

    print(f"\n  - Found ConceptNet Relations:\n  " + "\n  ".join(found_relations))

    # 4. Build and Refine Explanation
    final_explanation = build_and_refine_explanation(user_question, found_relations)
    print(f"\n🧠 Explanation:\n{final_explanation}")
    print("="*50)

In [14]:

# --- Example : A Different Domain ---
explainable_ai_tutor("Why should I drink water when I exercise?")

USER QUESTION: Why should I drink water when I exercise?
-> Generating LLM answer for: 'Why should I drink water when I exercise?'

🤖 LLM Answer:
Drinking water when you exercise is essential for several reasons:

1. **Prevents Dehydration**: Exercise causes you to lose water through sweat, and dehydration can lead to fatigue, dizziness, and even heatstroke. Drinking water helps replace the lost fluids and prevents dehydration.
2. **Boosts Performance**: Proper hydration improves physical performance by allowing your muscles to function efficiently. It helps regulate body temperature, reduces muscle cramping, and improves endurance.
3. **Reduces Muscle Soreness**: Drinking water after exercise can help reduce muscle soreness and inflammation by flushing out waste products and toxins.
4. **Maintains Electrolyte Balance**: Water helps maintain the balance of electrolytes (such as sodium, potassium, and calcium) in your body, which is essential for proper muscle and nerve function.
5. **S